<a href="https://colab.research.google.com/github/okana2ki/intro-to-AI/blob/main/student_faculty_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# sentence-transformers日本語版
https://github.com/sonoisa/sentence-transformers

以下、このセクションのプログラムは、↑のサイトからのコピーです。技術情報が十分には開示されていませんが、それなりの精度で埋め込みベクトルに変換できているのではないかと思います。変換精度評価と改良は今後の課題。その際に、参考にするサイトを下記にメモ：

https://www.sbert.net/

https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part18.html

https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part9.html

In [ ]:
!pip install -qU transformers fugashi ipadic

In [ ]:
from transformers import BertJapaneseTokenizer, BertModel
import torch


class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest",
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)

In [ ]:
model = SentenceBertJapanese("sonoisa/sentence-bert-base-ja-mean-tokens-v2")

# 割り当てタスク用プログラム

## 教員情報の処理


### 1. 教員CSVファイルからsentencesを作成

In [ ]:
import pandas as pd

# CSVファイルの読み込み
faculty_csv_path = '/content/drive/MyDrive/Colab_files/faculty.csv'  # 適切なパスに変更してください
faculty_df = pd.read_csv(faculty_csv_path)
# faculty_df = pd.read_csv(faculty_csv_path, encoding='shift_jis')

# NaNやNoneを含む可能性がある行を削除 <- エラー対策
faculty_df = faculty_df.dropna(subset=['description'])

# description列からsentencesリストを生成
fa_sentences = faculty_df['description'].tolist()

# sentencesの各要素が文字列であることを確認 <- エラー対策
fa_sentences = [str(sentence) for sentence in fa_sentences]

# sentencesリストを作成
# fa_sentences = faculty_df['description'].tolist()

In [ ]:
print(fa_sentences)

### 2. 埋め込みを生成

In [ ]:
fa_embeddings = model.encode(fa_sentences)

### Google Driveにembeddingsを保存するプログラム

In [ ]:
import os

# 保存するディレクトリのパスを指定（存在しない場合は作成）
save_dir = '/content/drive/MyDrive/Colab_files'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# ファイルに保存
file_path = os.path.join(save_dir, 'fa_embeddings.pt')
torch.save(fa_embeddings, file_path)

print(f'embeddingsが{file_path}に保存されました。')

### Google Driveからembeddingsを読み出すプログラム

In [ ]:
import os

# 読み出すファイルのパスを指定
file_path = '/content/drive/MyDrive/Colab_files/fa_embeddings.pt'

# ファイルが存在するか確認
if os.path.exists(file_path):
    # ファイルから読み出し
    fa_embeddings = torch.load(file_path)
    print(f'embeddingsが{file_path}から読み出されました。サイズ: {fa_embeddings.size()}')
else:
    print(f'{file_path}が見つかりません。ファイルパスを確認してください。')


In [ ]:
print(type(fa_sentences)) # debug用
fa_embeddings.shape # debug用

### 3. 主成分分析（PCA）を使用して埋め込みベクトルを2次元に削減し、結果を可視化

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# PCAで2次元に削減
pca = PCA(n_components=2)
X_pca = pca.fit_transform(fa_embeddings)  # embeddingsはmodel.encodeの出力

# 可視化
plt.figure(figsize=(10, 10))
for idx, point in enumerate(X_pca):
    plt.scatter(point[0], point[1])
    plt.text(point[0], point[1], faculty_df['name'].iloc[idx], fontsize=9)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('PCA Visualization of Faculty Descriptions')
plt.show()

### 4. コサイン類似度で全文書（全教員）との類似度を計算した後で、全文書を類似度の降順で並べて、faculty.csvのname列の名前で表示する

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 全文書とのコサイン類似度を計算
# ここでは、全文書に対して一度に類似度を計算します
# 例として、最初の文書をクエリとして使用します
# query_embedding = fa_embeddings[0].reshape(1, -1)  # 最初の文書の埋め込みベクトル
query_embedding = fa_embeddings[1].reshape(1, -1)
similarity_scores = cosine_similarity(query_embedding, fa_embeddings).flatten()

# 類似度スコアに基づいて文書のインデックスを降順にソート
sorted_doc_indices = similarity_scores.argsort()[::-1]

# 類似度の降順に文書（名前）と類似度スコアを表示
print("全文書を類似度の降順で表示:")
for idx in sorted_doc_indices:
    print(f"{faculty_df['name'].iloc[idx]}: {similarity_scores[idx]:.4f}")

## 学生情報の処理



### 1. 学生CSVファイルからsentencesを作成

In [ ]:
import pandas as pd

# CSVファイルの読み込み
student_csv_path = '/content/drive/MyDrive/Colab_files/2023students.csv'  # 適切なパスに変更してください
student_df = pd.read_csv(student_csv_path)
# faculty_df = pd.read_csv(faculty_csv_path, encoding='shift_jis')

# NaNやNoneを含む可能性がある行を削除 <- エラー対策
student_df = student_df.dropna(subset=['description'])

# description列からsentencesリストを生成
st_sentences = student_df['description'].tolist()

# sentencesの各要素が文字列であることを確認 <- エラー対策
st_sentences = [str(sentence) for sentence in st_sentences]

In [ ]:
print(st_sentences)

### 2. 埋め込みを生成

In [ ]:
st_embeddings = model.encode(st_sentences)

### Google Driveにembeddingsを保存するプログラム

In [ ]:
import os

# 保存するディレクトリのパスを指定（存在しない場合は作成）
save_dir = '/content/drive/MyDrive/Colab_files'  # 'your_directory'は適宜変更してください
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# ファイルに保存
file_path = os.path.join(save_dir, 'st_embeddings.pt')
torch.save(st_embeddings, file_path)

print(f'embeddingsが{file_path}に保存されました。')

### Google Driveからembeddingsを読み出すプログラム

In [ ]:
import os

# 読み出すファイルのパスを指定
file_path = '/content/drive/MyDrive/Colab_files/st_embeddings.pt'

# ファイルが存在するか確認
if os.path.exists(file_path):
    # ファイルから読み出し
    st_embeddings = torch.load(file_path)
    print(f'embeddingsが{file_path}から読み出されました。サイズ: {st_embeddings.size()}')
else:
    print(f'{file_path}が見つかりません。ファイルパスを確認してください。')

In [ ]:
st_embeddings.shape # debug用

### 3. 主成分分析（PCA）を使用して埋め込みベクトルを2次元に削減し、結果を可視化

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# PCAで2次元に削減
pca = PCA(n_components=2)
X_pca = pca.fit_transform(st_embeddings)  # st_embeddingsはmodel.encodeの出力

# 可視化
plt.figure(figsize=(10, 10))
for idx, point in enumerate(X_pca):
    plt.scatter(point[0], point[1])
    plt.text(point[0], point[1], student_df['id'].iloc[idx], fontsize=9)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('PCA Visualization of Students Descriptions')
plt.show()

## 学生の教員への割り当て

### 1. コサイン類似度で全学生-全教員間の類似度を計算
*   各学生について降順でソート（類似した順に全教員を表示）
*   各教員について降順でソート（類似した順に全学生を表示）



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Convert tensors to numpy arrays for compatibility with sklearn
fa_embeddings_np = fa_embeddings.numpy()
st_embeddings_np = st_embeddings.numpy()

# Calculate cosine similarity
# The result will be a matrix of shape [98, 23] where each row corresponds to a student and each column to a faculty
cos_sim = cosine_similarity(st_embeddings_np, fa_embeddings_np)

# Sort similarities for each student
student_sorted_indices = np.argsort(-cos_sim, axis=1)  # Sort indices in descending order of similarity for each student

# Sort similarities for each faculty
faculty_sorted_indices = np.argsort(-cos_sim.T, axis=1)  # Sort indices in descending order of similarity for each faculty

student_sorted_indices, faculty_sorted_indices

GPT-4への指示：

上記の結果を、次の2つのファイルに書き加えて下さい。

2023students.csv: このファイルのsimilarity列に、類似した順に全ファカルティを表示して下さい。ファカルティはfaculty.csvのname列の名前（faculty.csvの掲載順とfa_embeddingsの掲載順は一致しています）で表示して下さい。

faculty.csv: このファイルのsimilarity列に、類似した順に全学生を表示して下さい。学生は2023students.csvのid列の番号（これst_はembeddingsの順番と一致しています）で表示して下さい。

In [ ]:
import pandas as pd

# Load the CSV files
student_df = pd.read_csv('/content/drive/MyDrive/Colab_files/2023students.csv')
faculty_df = pd.read_csv('/content/drive/MyDrive/Colab_files/faculty.csv')

# Verify the content of the files
student_df.head(), faculty_df.head()

In [ ]:
# Mapping faculty indices to names
faculty_names = faculty_df['name'].tolist()

# Updating the 'similarity' column for students with faculty names in descending similarity order
student_df['similarity'] = ['; '.join([faculty_names[i] for i in row]) for row in student_sorted_indices]

# Mapping student indices to their IDs
student_ids = student_df['id'].tolist()

# Updating the 'similarity' column for faculties with student IDs in descending similarity order
faculty_df['similarity'] = ['; '.join([str(student_ids[i]) for i in row]) for row in faculty_sorted_indices]

# Save the updated dataframes to new CSV files
updated_students_csv_path = '/content/drive/MyDrive/Colab_files/updated_2023students.csv'
updated_faculty_csv_path = '/content/drive/MyDrive/Colab_files/updated_faculty.csv'

student_df.to_csv(updated_students_csv_path, index=False)
faculty_df.to_csv(updated_faculty_csv_path, index=False)

updated_students_csv_path, updated_faculty_csv_path

### 2. 割り当てアルゴリズム

何らかの損失関数を定義して最適化するのが正攻法だが、タスクの性質上、そこまでの精度は必要ないと判断し、計算量が小さい決定的なアルゴリズムを作ることにした。

GPT-4への指示（アルゴリズムは人が考え、実装は生成AIに任せるという分業スタイル）：

---
次の手順のプログラムを作成して下さい。
1. 全学生-全ファカルティ間の埋め込みのコサイン類似度を計算する。
2. すべてを類似度の降順で一列にならべる（98x23の類似度が一列に並ぶ）。
3. 次の手順で学生-ファカルティの1対1のペアを98組作る。結果として学生は1つのペアに属する。ファカルティは4または5のペアに属する。各ファカルティの所属ペア数をゼロに初期設定する。
4. ソート列から先頭の1ペアを取り出す。これをペアとして登録する。
5. 4.でペアとなった学生が属するペアの類似度データ全てをソート列から削除する。
6. 4.でペアとなったファカルティの所属ペア数を1増やす。その結果所属ペア数が5になった場合、そのファカルティが属するペアの類似度データ全てをソート列から削除する。
7. ソート列が空になったら終わり。空でなければ4.に戻る
---
実行の結果、3名以下の学生しか担当しない教員が生じる（できるだけ均等に割り振るという目的から外れる）ことが分かったので、アルゴリズムを再考し、以下の通り、GPT-4に指示：

---
属するペア数が少ないファカルティが存在することを防ぐように、アルゴリズムを改良しました。次の手順のプログラムを作成して下さい。
1. 全学生-全ファカルティ間の埋め込みのコサイン類似度を計算する。
2. すべてを類似度の降順で一列にならべる（98x23の類似度が一列に並ぶ）。
3. 次の手順で学生-ファカルティの1対1のペアを98組作る。結果として学生は1つのペアに属する。ファカルティは4または5のペアに属する。各ファカルティの所属ペア数をゼロに初期設定する。「5個のペアに属するファカルティ数」をゼロに初期設定する。
4. ソート列から先頭の1ペアを取り出す。これをペアとして登録する。
5. 4.でペアとなった学生が属するペアの類似度データ全てをソート列から削除する。
6. 4.でペアとなったファカルティの所属ペア数を1増やす。その結果所属ペア数が5になった場合、そのファカルティが属するペアの類似度データ全てをソート列から削除する。
所属ペア数が5になった場合は、「5個のペアに属するファカルティ数」を1増やす。これが6になった場合は、所属ペア数が4であるファカルティが属するペアの類似度データ全てをソート列から削除する。
7. ソート列が空になったら終わり。空でなければ4.に戻る。

In [ ]:
# Step 1: Re-calculate cosine similarity
cos_sim = cosine_similarity(st_embeddings_np, fa_embeddings_np)

# Step 2: Flatten and sort by similarity in descending order
cos_sim_flat = cos_sim.flatten()
sorted_indices = np.argsort(-cos_sim_flat)
sorted_flat_indices = sorted_indices

# Convert flat indices to 2D indices (student, faculty)
num_students, num_faculties = cos_sim.shape
student_indices, faculty_indices = np.unravel_index(sorted_flat_indices, (num_students, num_faculties))

# Step 3: Initialize pair counts and the counter for faculties with 5 pairs
faculty_pair_counts = np.zeros(num_faculties, dtype=int)
faculties_with_5_pairs = 0

# Initialize lists to store final pairs
final_pairs = []

# Track used students and faculties to remove them from consideration as needed
used_students = set()
used_faculties = set()

while sorted_flat_indices.size > 0:
    for i, flat_index in enumerate(sorted_flat_indices):
        student_index, faculty_index = np.unravel_index(flat_index, (num_students, num_faculties))

        # Skip if student or faculty already used
        if student_index in used_students or faculty_index in used_faculties:
            continue

        # Step 4: Register the pair
        final_pairs.append((student_index, faculty_index))
        used_students.add(student_index)
        faculty_pair_counts[faculty_index] += 1

        # Step 5 & 6: Remove used student and update faculty pair count
        if faculty_pair_counts[faculty_index] == 5:
            faculties_with_5_pairs += 1
            used_faculties.add(faculty_index)

        # Break the loop after registering a pair to update the sorting
        break

    # Update sorted indices to remove used students and faculties
    remaining_indices = [i for i, (s_i, f_i) in enumerate(zip(student_indices, faculty_indices))
                         if s_i not in used_students and f_i not in used_faculties]
    sorted_flat_indices = sorted_flat_indices[remaining_indices]
    student_indices, faculty_indices = np.unravel_index(sorted_flat_indices, (num_students, num_faculties))

    # Step 6: Check if it's time to remove faculties with 4 pairs
    if faculties_with_5_pairs == 6:
        for faculty_index in range(num_faculties):
            if faculty_pair_counts[faculty_index] == 4:
                used_faculties.add(faculty_index)  # Remove faculties with 4 pairs

        # Update sorted indices to remove faculties with 4 pairs
        remaining_indices = [i for i, (s_i, f_i) in enumerate(zip(student_indices, faculty_indices))
                             if s_i not in used_students and f_i not in used_faculties]
        sorted_flat_indices = sorted_flat_indices[remaining_indices]
        student_indices, faculty_indices = np.unravel_index(sorted_flat_indices, (num_students, num_faculties))

# Verify final pairings
len(final_pairs), final_pairs[:10]  # Show first 10 pairs for brevity


GPT-4への指示：

このペアリングの結果を次の2つのファイルに書き加えて下さい。

updated_2023students.csv: このファイルのpair列に、ペアとなったファカルティを表示して下さい。ファカルティはfaculty.csvのname列の名前（faculty.csvの掲載順とfa_embeddingsの掲載順は一致しています）で表示して下さい。

updated_faculty.csv: このファイルのpair列に、ペアとなった学生のリストを表示して下さい。学生は2023students.csvのid列の番号（これst_はembeddingsの順番と一致しています）で表示して下さい。

In [ ]:
# Re-import pandas after reset
import pandas as pd

# Load the updated CSV files again
student_df = pd.read_csv('/content/drive/MyDrive/Colab_files/updated_2023students.csv')
faculty_df = pd.read_csv('/content/drive/MyDrive/Colab_files/updated_faculty.csv')

# Extract the final pairs from the previous output
# final_pairs = [
    # (97, 3), (21, 9), (40, 1), (2, 3), (50, 18), (0, 18), (56, 18), (81, 18), (77, 0), (10, 18),
    # This list should continue with all pairs generated in the final output
# ]

# Reset 'pair' columns in both DataFrames
student_df['pair'] = ''
faculty_df['pair'] = [[] for _ in range(len(faculty_df))]

# Update the 'pair' column in students_df with the names of paired faculties
for student_index, faculty_index in final_pairs:
    faculty_name = faculty_df.iloc[faculty_index]['name']
    student_df.at[student_index, 'pair'] = faculty_name

# Update the 'pair' column in faculty_df with the list of paired students
for student_index, faculty_index in final_pairs:
    student_id = student_df.iloc[student_index]['id']
    if type(faculty_df.at[faculty_index, 'pair']) == list:
        faculty_df.at[faculty_index, 'pair'].append(student_id)
    else:
        faculty_df.at[faculty_index, 'pair'] = [student_id]

# Convert lists in 'pair' column of faculty_df to a semicolon-separated string for consistency
faculty_df['pair'] = faculty_df['pair'].apply(lambda x: '; '.join(map(str, x)) if isinstance(x, list) else x)

# Save the updated dataframes back to new CSV files
updated_students_csv_path = '/content/drive/MyDrive/Colab_files/final_updated_2023students.csv'
updated_faculty_csv_path = '/content/drive/MyDrive/Colab_files/final_updated_faculty.csv'

student_df.to_csv(updated_students_csv_path, index=False)
faculty_df.to_csv(updated_faculty_csv_path, index=False)

updated_students_csv_path, updated_faculty_csv_path